In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm.auto import tqdm
import torch
import cv2
from glob import glob

In [ ]:
!cp -r ../input/yolov5git/yolov5-master/* .

In [ ]:
DATA_ROOT_PATH = 'test_images'
WEIGHTS = '../input/yolov5m-1024-tmp/best.pt'
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )

In [ ]:
# https://www.kaggle.com/its7171/2class-object-detection-inference/
def mk_images(video_name, video_labels, video_dir, out_dir, only_with_impact=True):
    video_path=f"{video_dir}/{video_name}"
    video_name = os.path.basename(video_path)
    vidcap = cv2.VideoCapture(video_path)
    if only_with_impact:
        boxes_all = video_labels.query("video == @video_name")
        print(video_path, boxes_all[boxes_all.impact == 1.0].shape[0])
    else:
        print(video_path)
    frame = 0
    while True:
        it_worked, img = vidcap.read()
        if not it_worked:
            break
        frame += 1
        if only_with_impact:
            boxes = video_labels.query("video == @video_name and frame == @frame")
            boxes_with_impact = boxes[boxes.impact == 1.0]
            if boxes_with_impact.shape[0] == 0:
                continue
        img_name = f"{video_name}_frame{frame}"
        image_path = f'{out_dir}/{video_name}'.replace('.mp4',f'_{frame}.png')
        _ = cv2.imwrite(image_path, img)

In [ ]:
out_dir = DATA_ROOT_PATH
if not os.path.exists(out_dir):
    !mkdir -p $out_dir
    video_dir = '/kaggle/input/nfl-impact-detection/test'
    uniq_video = [path.split('/')[-1] for path in glob(f'{video_dir}/*.mp4')]
    for video_name in uniq_video:
        mk_images(video_name, pd.DataFrame(), video_dir, out_dir, only_with_impact=False)
        # for fast commit
        if len(uniq_video) == 6:
            break

In [ ]:
from utils.datasets import LoadImages
from models.experimental import attempt_load
from utils.torch_utils import time_synchronized
from utils.general import *
from utils.plots import plot_one_box

In [ ]:
model = attempt_load(weights=WEIGHTS, map_location=device)  # load FP32 model

In [ ]:
names = model.module.names if hasattr(model, 'module') else model.names

In [ ]:
dataset = LoadImages(DATA_ROOT_PATH, img_size=1024)

In [ ]:
conf_thres = 0.25
iou_thres = 0.35
score_thres = 0.4

In [ ]:
result_image_ids = []
results_boxes = []
results_scores = []

t0 = time.time()
for path, img, im0s, vid_cap in dataset:
    img = torch.from_numpy(img).to(device).float()
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    t1 = time_synchronized()
    pred = model(img, augment=False)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes=None, agnostic=False)
    t2 = time_synchronized()

    image_id = Path(path).name
    
    # Process detections
    for i, det in enumerate(pred):  # detections per image
        p, s, im0 = Path(path), '', im0s

        det = det[det[:, 5] > 0]  # filter out 0
        
        if len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
            
            det[:, 2] = det[:, 2] - det[:, 0]
            det[:, 3] = det[:, 3] - det[:, 1]
            det[:, 0] = det[:, 0].clip(min=0, max=1280-1)
            det[:, 2] = det[:, 2].clip(min=0, max=1280-1)
            det[:, 1] = det[:, 1].clip(min=0, max=720-1)
            det[:, 3] = det[:, 3].clip(min=0, max=720-1)
            
            result_image_ids += [image_id]*len(det)
            results_boxes.append(det[:, :4].cpu().data.numpy())
            results_scores.append(det[:, 4].cpu().data.numpy())

            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += '%g %ss, ' % (n, names[int(c)])  # add to string

        # Print time (inference + NMS)
        print('%sDone. (%.3fs)' % (s, t2 - t1))

print('Done. (%.3fs)' % (time.time() - t0))

In [ ]:
box_df = pd.DataFrame(np.concatenate(results_boxes), columns=['left', 'top', 'width', 'height'])
test_df = pd.DataFrame({'scores':np.concatenate(results_scores), 'image_name':result_image_ids})
test_df = pd.concat([test_df, box_df], axis=1)

test_df = test_df[test_df.scores > score_thres]

test_df.shape

In [ ]:
#gameKey,playID,view,video,frame,left,width,top,height
#57590,3607,Endzone,57590_003607_Endzone.mp4,1,1,1,1,1
test_df['gameKey'] = test_df.image_name.str.split('_').str[0].astype(int)
test_df['playID'] = test_df.image_name.str.split('_').str[1].astype(int)
test_df['view'] = test_df.image_name.str.split('_').str[2]
test_df['frame'] = test_df.image_name.str.split('_').str[3].str.replace('.png','').astype(int)
test_df['video'] = test_df.image_name.str.rsplit('_',1).str[0] + '.mp4'
test_df = test_df[["gameKey","playID","view","video","frame","left","width","top","height"]]
test_df

In [ ]:
!mv * /tmp/

In [ ]:
import nflimpact
env = nflimpact.make_env()
env.predict(test_df) # d